In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
import json

# 検索結果を返す関数の作成
def get_search_result(keyword, when="1", unit="m"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result)

In [3]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [4]:
# 言語モデルが直接回答できる質問
question = "日本の首都はどこにありますか？"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))

{'_request_id': 'req_08f9b4d6839d17ae91a2af45ed63139f',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='日本の首都は東京にあります。東京は日本の政治、経済、文化の中心地であり、国内外から多くの人々が集まる都市です。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))],
 'created': 1747274516,
 'id': 'chatcmpl-BXICyAvDUvCFlSPkL33JN7cFj4MkX',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_0392822090',
 'usage': CompletionUsage(completion_tokens=42, prompt_tokens=91, total_tokens=133, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [5]:
# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))

{'_request_id': 'req_8fb522c07c7ca065e051f42685d7f7b5',
 'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_IOYLqJMGhWPDqwLmB3AzQnRu', function=Function(arguments='{"keyword":"東京駅\u3000イベント","when":30,"unit":"d"}', name='get_search_result'), type='function')], annotations=[]))],
 'created': 1747274686,
 'id': 'chatcmpl-BXIFi1fPpvWPl4Cw3bOBeNZ7CLBVw',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_0392822090',
 'usage': CompletionUsage(completion_tokens=27, prompt_tokens=97, total_tokens=124, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))}


In [6]:
# モデルがツール呼出と判断した
if response.choices[0].finish_reason == 'tool_calls':
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")

    # 引数の取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数の実行
    function_response = globals()[function_name](**arguments)
    print("関数の実行結果：")
    pprint(json.loads(function_response))

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response_after_tool_call.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response.choices[0].message.content.strip())

関数名：get_search_result
引数：{'keyword': '東京駅\u3000イベント', 'when': 30, 'unit': 'd'}
関数の実行結果：
{'result': [{'title': '1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました'},
            {'title': '1929年（昭和4年）八重洲口が開設されました'},
            {'title': '2014年（平成26年）東京駅開業100周年を迎えました'}]}
言語モデルからの回答：
最近1ヶ月以内の東京駅のイベントに関する検索結果は以下の通りです：

1. **1914年（大正3年）** 中央停車場改め「東京駅」として開業を開始しました。
2. **1929年（昭和4年）** 八重洲口が開設されました。
3. **2014年（平成26年）** 東京駅開業100周年を迎えました。

これらの情報は東京駅の歴史に関連するイベントの概要ですが、具体的な最近のイベント情報については、公式サイトや関連のニュースメディアでの確認が必要です。
